# Säulen- und Balkendiagramme
Säulen- und Balkendiagramme sind die Diagramme zur vergleichenden Darstellung. Die Höhe, manchmal auch die Fläche, verhält sich dabei proportional zur Ausprägung eines Merkmals.

In [ ]:
import pandas as pd
import plotly.express as px
from ipywidgets import interact

from tui_dsg.datasets import disney_plus_titles_extended_path

## Vorverarbeitung
Laden Sie den Datensatz [Disney+ Movies and TV Shows](https://www.kaggle.com/code/werador/disney-data-analysis/data).

In [ ]:
df = pd.read_csv(disney_plus_titles_extended_path, parse_dates=['date_added'])
df = df[['type', 'country', 'release_year', 'duration']]

df

Die nachfolgende Zelle reduziert jedes Land auf die Anzahl der Filme und Serien.

In [ ]:
df2 = df[['type']].join(
    df['country'].str.split(', ', expand=True).stack().reset_index(level=1, drop=True).rename('country')
)
df3 = pd.merge(
    df2[df2['type'] == 'TV Show'].groupby('country').count().rename(columns={'type': 'movies'}).reset_index(),
    df2[df2['type'] == 'Movie'].groupby('country').count().rename(columns={'type': 'tvshows'}).reset_index(),
    on='country'
)

Die nachfolgende Zelle berechnet die durchschnittliche Filmlänge nach Jahr.

In [ ]:
df4 = df[df['type'] == 'Movie'][['release_year', 'duration']]
df4['duration_in_minutes'] = df4['duration'].map(lambda x: int(x[:-4]))

df5 = pd.merge(
    df4.groupby('release_year')['duration_in_minutes'].mean().rename('duration').reset_index(),
    df4.groupby('release_year')['duration_in_minutes'].sem().rename('sem').reset_index(),
    on='release_year'
)

## Ein simples Säulendiagramm
Anschließend können die Werte als Säulendiagramm dargestellt werden. Zusätzlich können die Achsen durch die booleschen Paramter `log_y` und `log_x` logarithmisch skaliert werden.

In [ ]:
@interact(log_y=False)
def _(log_y):
    return px.bar(df3, x='country', y='movies', log_y=log_y)

## Balkendiagramme
Balkendiagramme werden wie Säulendiagramme erstellt. Dabei wird der Parameter `orientation` auf `"h"` gesetzt und die Werte für die x- und y-Achse vertauscht.

In [ ]:
px.bar(df3, x='movies', y='country', orientation='h')

## Gruppen und Stapel
Zusätzlich lassen sich verschiedene Werte durch den Paramter `barmode` entweder gruppieren (`group`) oder stapeln (`stack`). Der y-Achse wird dabei eine Liste von Spaltennamen zugeordnet.

In [ ]:
@interact(barmode=['group', 'stack'])
def _(barmode):
    return px.bar(df3, x='country', y=['movies', 'tvshows'], barmode=barmode, log_y=True)

## Fehlerbalken
Fehlerbalken lassen sich durch die Parameter `error_y` und `error_x` hinzufügen.

In [ ]:
px.bar(df5, x='release_year', y='duration', error_y='sem')